<a href="https://colab.research.google.com/github/tnr1337/MyProject/blob/main/BTCindikat%C3%B6rlerleAlSat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **BTC İNDİKATÖRLERLE AL SAT UYGULAMASI**



Bu projemizde kullanacağımız temel kütüphaneleri yüklüyoruz ve projemize dahil ediyoruz. İlk olarak, pip komutumuzla gerekli paketleri yüklüyoruz. Burada yfinance, pandas, numpy, plotly, dash ve datetime paketlerini indiriyoruz.

Ardından, bu kütüphaneleri projemize import ediyoruz. yfinance'ı 'yf' kısa adıyla çağırıyoruz, bu bize finansal verileri çekmekte yardımcı olacak. pandas ve numpy, veri işleme ve analiz için vazgeçilmez araçlarımız. plotly ve onun alt modülleri, interaktif grafikler oluşturmamızı sağlayacak. dash ve ilgili modülleri ise web uygulamamızı oluşturmak için kullanacağız. Son olarak, datetime modülü tarih ve zaman işlemlerimiz için gerekli.

Bu imports sayesinde, projemizin temel yapı taşlarını oluşturmuş oluyoruz ve artık veri çekme, işleme, görselleştirme ve web uygulaması oluşturma gibi ana işlevleri gerçekleştirebileceğiz.

In [ ]:
!pip install yfinance pandas numpy plotly dash datetime
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import datetime


Bu 'get_data()' fonksiyonu, Bitcoin'in son bir yıllık fiyat verilerini çekmek için tasarlanmıştır. Fonksiyon, şu anki tarihi bitiş tarihi olarak alır ve başlangıç tarihini bir yıl öncesi olarak hesaplar. Ardından, yfinance kütüphanesini kullanarak BTC-USD sembolü için günlük verileri indirir ve bu verileri geri döndürür. Bu sayede, her çağrıldığında en güncel bir yıllık Bitcoin verilerini elde ederiz, bu da grafiklerimiz ve analizlerimiz için temel oluşturur.

In [ ]:
def get_data():
    end_date = datetime.datetime.now()
    start_date = end_date - datetime.timedelta(days=365)  # Son 1 yıllık veri
    data = yf.download('BTC-USD', start=start_date, end=end_date, interval='1d')
    return data

Bu kod bloğu, üç önemli teknik analiz indikatörünü hesaplamak için fonksiyonlar içerir. 'calculate_sma' fonksiyonu, Basit Hareketli Ortalama (SMA) hesaplar ve belirtilen dönem için kapanış fiyatlarının ortalamasını alır. 'calculate_ema' fonksiyonu, Üstel Hareketli Ortalama (EMA) hesaplar, bu da daha yeni verilere daha fazla ağırlık verir. Son olarak, 'calculate_rsi' fonksiyonu Göreceli Güç İndeksi (RSI) hesaplar, bu indikatör fiyat momentumunu ölçer ve aşırı alım veya satım koşullarını belirlemede kullanılır. Bu fonksiyonlar, Bitcoin fiyat hareketlerini analiz etmek ve potansiyel alım-satım sinyalleri üretmek için kullanılacaktır.

In [ ]:
# İndikatör hesaplama fonksiyonları
def calculate_sma(data, period):
    return data['Close'].rolling(window=period).mean()

def calculate_ema(data, period):
    return data['Close'].ewm(span=period, adjust=False).mean()

def calculate_rsi(data, period):
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))


Bu kod bloğu iki önemli bölümden oluşuyor. İlk olarak, 'generate_signals' fonksiyonu, 20 ve 50 günlük Basit Hareketli Ortalamalar (SMA) kullanarak alım-satım sinyalleri üretir. 20 günlük SMA'nın 50 günlük SMA'yı yukarı kesmesi durumunda alım sinyali, aşağı kesmesi durumunda ise satım sinyali oluşturur. İkinci bölümde, Dash uygulaması başlatılır ve temel düzeni oluşturulur. Uygulama, bir başlık, canlı güncellenen bir grafik ve her dakikada bir yenileme yapan bir zamanlayıcı içerir. Bu yapı, kullanıcıya interaktif ve sürekli güncellenen bir Bitcoin fiyat grafiği sunmayı amaçlar.

In [ ]:
# Al-sat sinyalleri oluşturma fonksiyonu
def generate_signals(data):
    sma_20 = calculate_sma(data, 20)
    sma_50 = calculate_sma(data, 50)

    buy_signals = (sma_20 > sma_50) & (sma_20.shift(1) <= sma_50.shift(1))
    sell_signals = (sma_20 < sma_50) & (sma_20.shift(1) >= sma_50.shift(1))

    return buy_signals, sell_signals

# Dash uygulamasını başlat
app = dash.Dash(__name__)

# Uygulama düzeni
app.layout = html.Div([
    html.H1("Bitcoin Fiyat Grafiği ve İndikatörler"),
    dcc.Graph(id='live-graph'),
    dcc.Interval(
        id='interval-component',
        interval=60*1000,  # Her 1 dakikada bir güncelle
        n_intervals=0
    )
])

Bu kod bloğu, Dash uygulamasının kalbini oluşturan 'update_graph' callback fonksiyonunu içerir. Bu fonksiyon, her bir dakikada bir çağrılarak grafiği güncellemekten sorumludur. İlk olarak, güncel Bitcoin verilerini çeker ve bu veriler üzerinde SMA, EMA ve RSI indikatörlerini hesaplar. Ardından, önceden tanımladığımız alım-satım sinyallerini oluşturur.

Grafik oluşturma kısmında, plotly kütüphanesi kullanılarak detaylı bir görselleştirme yapılır. Üst kısımda bir mum grafiği yer alır ve üzerine SMA ve EMA çizgileri eklenir. Alım-satım sinyalleri, grafikte üçgen işaretlerle gösterilir. Alt kısımda ise RSI göstergesi bulunur, 30 ve 70 seviyeleri aşırı alım ve satım bölgelerini belirtmek için çizilmiştir. Grafik, başlık, eksen etiketleri ve boyut ayarlarıyla son halini alır. Son olarak, uygulama debug modunda çalıştırılır. Bu yapı, kullanıcıya Bitcoin fiyatlarını, teknik indikatörleri ve alım-satım sinyallerini tek bir interaktif grafikte sunarak, kripto para piyasasını analiz etmek için güçlü bir araç sağlar.

In [ ]:
# Callback fonksiyonu
@app.callback(Output('live-graph', 'figure'),
              Input('interval-component', 'n_intervals'))
def update_graph(n):
    data = get_data()

    # İndikatörleri hesapla
    data['SMA20'] = calculate_sma(data, 20)
    data['SMA50'] = calculate_sma(data, 50)
    data['EMA20'] = calculate_ema(data, 20)
    data['RSI'] = calculate_rsi(data, 14)

    # Al-sat sinyallerini oluştur
    buy_signals, sell_signals = generate_signals(data)

    # Grafik oluştur
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                        vertical_spacing=0.1, row_heights=[0.7, 0.3])

    # Mum grafiği
    fig.add_trace(go.Candlestick(x=data.index,
                open=data['Open'], high=data['High'],
                low=data['Low'], close=data['Close'],
                name='BTC-USD'), row=1, col=1)

    # SMA ve EMA çizgileri
    fig.add_trace(go.Scatter(x=data.index, y=data['SMA20'], name='SMA20', line=dict(color='blue')), row=1, col=1)
    fig.add_trace(go.Scatter(x=data.index, y=data['SMA50'], name='SMA50', line=dict(color='red')), row=1, col=1)
    fig.add_trace(go.Scatter(x=data.index, y=data['EMA20'], name='EMA20', line=dict(color='green')), row=1, col=1)

    # Al-sat sinyalleri
    fig.add_trace(go.Scatter(x=data.index[buy_signals], y=data['Low'][buy_signals],
                             mode='markers', name='Al Sinyali',
                             marker=dict(symbol='triangle-up', size=10, color='green')), row=1, col=1)
    fig.add_trace(go.Scatter(x=data.index[sell_signals], y=data['High'][sell_signals],
                             mode='markers', name='Sat Sinyali',
                             marker=dict(symbol='triangle-down', size=10, color='red')), row=1, col=1)

    # RSI göstergesi
    fig.add_trace(go.Scatter(x=data.index, y=data['RSI'], name='RSI'), row=2, col=1)
    fig.add_hline(y=30, line_dash="dash", line_color="green", row=2, col=1)
    fig.add_hline(y=70, line_dash="dash", line_color="red", row=2, col=1)

    fig.update_layout(title='Bitcoin Fiyat Grafiği ve İndikatörler',
                      xaxis_title='Tarih',
                      yaxis_title='Fiyat (USD)',
                      height=800)

    fig.update_yaxes(title_text="RSI", row=2, col=1)

    return fig

if __name__ == '__main__':
    app.run_server(debug=True)